# 473. Matchsticks to Square

## Topic Alignment
- Bitmask DFS is useful for combinatorial search spaces in hardware feature selection or distributed job scheduling where subsets must satisfy length constraints.

## Metadata 摘要
- Source: https://leetcode.com/problems/matchsticks-to-square/
- Tags: DFS, Backtracking, Bitmask
- Difficulty: Medium
- Priority: Medium

## Problem Statement 原题描述
You are given an integer array matchsticks where matchsticks[i] is the length of the i-th matchstick. You want to use all the matchsticks to make one square. You should not break any stick, but you can link them together, and each matchstick must be used exactly one time.

Return true if you can make this square and false otherwise.

## Progressive Hints
- Hint 1: The perimeter must be divisible by 4; otherwise return false immediately.
- Hint 2: Sort matchsticks in descending order to prune faster.
- Hint 3: Use DFS to assign sticks to sides, tracking consumed sticks via bitmask or visited array.

## Solution Overview
Sort sticks descending, compute side length = sum(matchsticks) // 4, and apply DFS with memoization on bitmasks. At each step we try to extend the current side length with unused sticks. Memoizing states (mask, current_side_length) avoids recomputation. When all sticks are used and the last side length is zero, formation succeeds.

## Detailed Explanation
1. Compute total = sum(matchsticks); if total % 4 != 0 or max stick > total/4, return False.
2. Sort matchsticks in descending order to fail early when large sticks do not fit.
3. Use DFS(mask, side_sum) where mask indicates which sticks are used, and side_sum is the length accumulated on the current side.
4. If mask == full_mask (all sticks used), success when side_sum == 0 because that means the last side is complete.
5. If side_sum == target, reset to 0 and start assembling the next side.
6. Iterate over sticks; if not used, try placing it if side_sum + stick <= target. Mark it used, recurse with updated mask and side_sum + stick, then backtrack.
7. Cache memo[(mask, side_sum)] to prevent redundant exploration.

## Complexity Trade-off Table
| Approach | Time | Space | Notes |
| --- | --- | --- | --- |
| DFS + bitmask memo | O(2^n * n) | O(2^n) | n <= 15, manageable with pruning |
| Backtracking without memo | Exponential | O(n) | May TLE on worst cases |

In [ ]:
from functools import lru_cache
from typing import List

class Solution:
    def makesquare(self, matchsticks: List[int]) -> bool:
        total = sum(matchsticks)
        if total % 4 != 0 or not matchsticks:
            return False
        target = total // 4
        matchsticks.sort(reverse=True)
        if matchsticks[0] > target:
            return False
        n = len(matchsticks)
        full_mask = (1 << n) - 1
        @lru_cache(None)
        def dfs(mask: int, side_sum: int) -> bool:
            if mask == full_mask:
                return side_sum == 0
            if side_sum == target:
                return dfs(mask, 0)
            for i in range(n):
                if mask & (1 << i):
                    continue
                stick = matchsticks[i]
                new_sum = side_sum + stick
                if new_sum > target:
                    continue
                if dfs(mask | (1 << i), new_sum):
                    return True
                if side_sum == 0:
                    break
            return False
        return dfs(0, 0)

In [ ]:
tests = [
    ([1,1,2,2,2], True),
    ([3,3,3,3,4], False),
    ([5,5,5,5,4,4,4,4,3,3,3,3], True),
    ([1,1,1,1,1,1,1,1,1,1,1,1,1,1,14], False)
]
solver = Solution()
for sticks, expected in tests:
    assert solver.makesquare(sticks[:]) == expected
print('All tests passed.')

## Complexity Analysis
- Time: O(2^n * n) in the worst case due to memoized subset exploration. n <= 15 fits comfortably.
- Space: O(2^n) for memo cache plus recursion depth up to n.

## Edge Cases & Pitfalls
- Sorting descending is crucial; without it the search may take significantly longer.
- Immediately reject when any stick exceeds target side length.
- Remember to allow repeated lengths; duplicates are part of the search space.

## Follow-up Variants
- Generalize to k-gon with perimeter divisible by k.
- Return one valid assignment of sticks to sides instead of boolean.
- Optimize for streaming matchstick updates where sticks arrive one by one.

## Takeaways
- Bitmask DFS coupled with memoization tames combinatorial explosion for n <= 15.
- Early pruning (divisibility test, sorted order) drastically reduces search.
- Recursion state should capture both used subset and progress on the current side.

## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| LC 698 | Partition to K Equal Sum Subsets | Bitmask DFS with memo |
| LC 1723 | Find Minimum Time to Finish All Jobs | Bitmask subset DP |
| LC 1986 | Minimum Number of Work Sessions to Finish the Tasks | DFS + bitmask |